In [73]:
def rotations(t):
 ''' Return list of rotations of input string t '''
 tt = t * 2 #duplicate string
 return [ tt[i:i+len(t)] for i in range(0, len(t)) ]
def bwm(t):
 ''' Return lexicographically sorted list of t’s rotations '''
 return sorted(rotations(t))
def bwtViaBwm(t):
 ''' Given T, returns BWT(T) by way of the BWM '''
 return ''.join(map(lambda x: x[-1], bwm(t))) #takes last column


In [74]:
t = 'abaaba$'
b = bwtViaBwm(t)
b


'abba$aa'

In [75]:
def rankBwt(bw):
     ''' Given BWT string bw, return parallel list of B-ranks. Also
     returns tots: map from character to # times it appears. '''
     tots = dict() #na kraju koliko ima ukupno kojih karaktera
     ranks = [] #do tog trenutka, ne cuva karakter samo rank
     for c in bw:
         if c not in tots: tots[c] = 0
         ranks.append(tots[c])
         tots[c] += 1
     return ranks, tots

In [76]:
ranks, tots = rankBwt(b)
print(zip(b, ranks)) # print characters of BWT(T) in order, along with rank
print(ranks)


[0, 0, 1, 1, 0, 2, 3]


In [77]:
def firstCol(tots):
     ''' Return map from character to the range of rows prefixed by
     the character. '''
     first = {}
     totc = 0
     for c, count in sorted(tots.items()):
         first[c] = totc #izmenila, bilo je(totc, totc + count), #c[char] je samo prvi indeks value-a od first[char] tj first[c][0] 
         totc += count
     return first


In [78]:
firstCol(tots) #dolar se nalazi u 0. vrsti, a od 1. do 4., b od 5. do 6.


{'$': 0, 'a': 1, 'b': 5}

In [79]:
# confirm that the representation of the first column above is sensible
print('\n'.join(bwm(t)))

$abaaba
a$abaab
aaba$ab
aba$aba
abaaba$
ba$abaa
baaba$a


In [80]:

def reverseBwt(bw):
     ''' Make T from BWT(T) '''
     ranks, tots = rankBwt(bw)
     first = firstCol(tots)
     rowi = 0 # start in first row
     t = '$' # start with rightmost character
     while bw[rowi] != '$':
         c = bw[rowi]
         t = c + t # prepend to answer #dodajemo na pocetak stringa tako da ce vec biti reverse-ovan
         # jump to row that starts with c of same rank
         rowi = first[c] + ranks[rowi] #first[c]-odatle pocinju redovi sa kar. c, i dodamo rank kar. c u t
     return t


In [81]:
reverseBwt(b)


'abaaba$'

In [82]:
reverseBwt(bwtViaBwm('In_the_jingle_jangle_morning$'))


'In_the_jingle_jangle_morning$'

In [83]:
#pravljenje BTW prek sufiksnog niza
def suffixArray(s):
 """ Given T return suffix array SA(T). We use Python's sorted
 function here for simplicity, but we can do better. """
 satups = sorted([(s[i:], i) for i in range(len(s))]) #uzmemo sve moguce sufikse i sortiramo, 
    #uz zadrzavanje inf o indeksu (i), tj poziciji stringa u tekstu
 # Extract and return just the offsets
# print(satups)
 return map(lambda x: x[1], satups) #vraca samo indekse
def bwtViaSa(t):
 """ Given T, returns BWT(T) by way of the suffix array. """
 bw = []
 sa = suffixArray(t)
 for si in sa:
     if si == 0: 
        bw.append('$')
     else: 
        bw.append(t[si-1])
 return ''.join(bw)# return string-ized version of list bw and suffixarray

In [84]:
list(suffixArray('ABAABA$'))


[6, 5, 2, 3, 0, 4, 1]

In [85]:
bwtViaSa('ABAABA$')


'ABBA$AA'

In [86]:
def lastCol(bw):
    ranks,tots=rankBwt(bw)
    return list(zip(ranks,bw))
    

In [87]:
lastCol('ABBA$AA')

[(0, 'A'), (0, 'B'), (1, 'B'), (1, 'A'), (0, '$'), (2, 'A'), (3, 'A')]

In [88]:
#FM


In [89]:
def tallyFun(c,bw):
    tally=dict()
    tots=dict()
    for char, row in c.items():
        tots[char]=0
        tally[char]={}
        for row in range (len(bw)):
            tally[char][row]=0
    print(tots)
    print(tally)
    for row in range (len(bw)):
        char = bw[row]
        print(char)
        tots[char] += 1 #prvo se inkrementira pa stavi!! nije kao rankovi
        for char in c.keys():
            tally[char][row]=tots[char]
    return tally  

In [176]:
def initFM(text):
    bw=bwtViaSa(text)
    sa=list(suffixArray(text))
    ranks,tots=rankBwt(bw)
    c=firstCol(tots)
    L=lastCol(bw)
    tally=tallyFun(c,bw)

In [174]:
def findPattern(text,pattern):
    #text='abaaba$'
    #text='banana$'
    #text=reverseBwt('GGGGGGTCAA$TAA')
    #bw=bwtViaSa(text)
    #sa=list(suffixArray(text))
    #print(sa)
    #print(bw)
    #ranks,tots=rankBwt(bw)
    #print(tots)
    #c=firstCol(tots)
    #print(c)
    #L=lastCol(bw)
    #print(L)
    #tally=tallyFun(c,bw)
    #print(tally)

    #pattern='aba'
    #pattern='ana'
    #pattern='GAGA'


    start=1
    end=len(bw)-1
    print(c)
    print(tally)
    for i in reversed(range(len(pattern))): #od poslednjeg indeksa do prvog
        char=pattern[i]
        if i==(len(pattern)-1):
            start = c[char]
            takeNext=False
            for j in c.keys():
                if takeNext:
                    end=c[j]-1
                    break
                if j==char:
                    takeNext=True       
        else:
            start=c[char]+tally[char][start-1]
            end=c[char]+tally[char][end]-1
        print(char)
        print(start)
        print(end)
        print('ok')
    print(start)
    print(end)

    positions=[]
    for i in range(start,end+1): #+1 jer range ne includuje poslendji navedeni indeks
        positions.append(sa[i])
    print(positions)
    return positions

In [93]:
match=2
gap=-6
mismatch=-4

In [173]:
def initScoringMatrix(Match,Gap,Mismatch):
    global match
    global gap
    global mismatch
    match=Match
    gap=Gap
    mismatch=Mismatch

In [172]:
def scoringMatrix(a, b):
 if a==b: return match# match
 if a=='_' or b=='_': return gap # indel
 return mismatch # mismatch

In [97]:
import numpy
def globalAlignment(x, y, s): #s je scoring matrica 
     D = numpy.zeros((len(x) + 1, len(y) + 1), dtype=int)

     for i in range(1, len(x) + 1):
         D[i,0] = D[i-1,0] + s(x[i-1], '_') 
     for j in range(1, len(y)+1):
         D[0,j] = D[0,j-1] + s('_', y[j-1])

     for i in range(1, len(x) + 1):
         for j in range(1, len(y) + 1):
             D[i,j] = max(D[i-1,j] + s(x[i-1], '_'),
                         D[i,j-1] + s('_', y[j-1]),
                         D[i-1,j-1] + s(x[i-1], y[j-1]))

     # function returns table and global alignment score
     #alignment score is in cell (n,m) of the matrix
     return D, D[len(x),len(y)]

In [98]:
def traceback(x,y,V,s):
 
     # initializing starting position cell(n,m)
     i=len(x)
     j=len(y)

     # initializing strings we use to represent alignments in x, y, edit transcript and global alignment
     ax, ay, am, tr = '', '', '', ''

     # exit condition is when we reach cell (0,0)
     while i > 0 or j > 0:

         # calculating diagonal, horizontal and vertical scores for current cell
         d, v, h = -100, -100, -100

         if i > 0 and j > 0:
             delta = 1 if x[i-1] == y[j-1] else 0
             d = V[i-1,j-1] + s(x[i-1], y[j-1]) # diagonal movement 
         if i > 0: 
            v = V[i-1,j] + s(x[i-1], '_') # vertical movement
         if j > 0: 
            h = V[i,j-1] + s('_', y[j-1]) # horizontal movement

         # backtracing to next (previous) cell
         if d >= v and d >= h:
             ax += x[i-1]
             ay += y[j-1]
             if delta == 1:
                 tr += 'M'
                 am += '|' #kao povezivanje dva stringa
             else:
                 tr += 'R'
                 am += ' '
             i -= 1
             j -= 1
         elif v >= h: #delecija
             ax += x[i-1]
             ay += '_'
             tr += 'D'
             am += ' '
             i -= 1
         else: #insercija
             ay += y[j-1]
             ax += '_'
             tr += 'I'
             am += ' '
             j -= 1

     alignment='\n'.join([ax[::-1], am[::-1], ay[::-1]])
     return alignment, tr[::-1]

In [99]:
x='TACGTCAGC'
y='TATGTCATGC'
D, alignmentScore = globalAlignment(x, y, scoringMatrix3)
alignment, transcript = traceback(x, y, D, scoringMatrix3)


In [100]:

print(alignment)
print(transcript)
print(D)
print(alignmentScore)

TACGTCA_GC
|| |||| ||
TATGTCATGC
MMRMMMMIMM
[[  0  -6 -12 -18 -24 -30 -36 -42 -48 -54 -60]
 [ -6   2  -4 -10 -16 -22 -28 -34 -40 -46 -52]
 [-12  -4   4  -2  -8 -14 -20 -26 -32 -38 -44]
 [-18 -10  -2   0  -6 -12 -12 -18 -24 -30 -36]
 [-24 -16  -8  -6   2  -4 -10 -16 -22 -22 -28]
 [-30 -22 -14  -6  -4   4  -2  -8 -14 -20 -26]
 [-36 -28 -20 -12 -10  -2   6   0  -6 -12 -18]
 [-42 -34 -26 -18 -16  -8   0   8   2  -4 -10]
 [-48 -40 -32 -24 -16 -14  -6   2   4   4  -2]
 [-54 -46 -38 -30 -22 -20 -12  -4  -2   0   6]]
6


In [102]:
x='TACGTCAGC'
y='TATGTCATGC'
D, alignmentScore = globalAlignment(x, y, scoringMatrix)
alignment, transcript = traceback(x, y, D, scoringMatrix)


In [103]:

print(alignment)
print(transcript)
print(D)
print(alignmentScore)

TACGTCA_GC
|| |||| ||
TATGTCATGC
MMRMMMMIMM
[[  0  -7 -14 -21 -28 -35 -42 -49 -56 -63 -70]
 [ -7   0  -7 -14 -21 -28 -35 -42 -49 -56 -63]
 [-14  -7   0  -7 -14 -21 -28 -35 -42 -49 -56]
 [-21 -14  -7  -3 -10 -17 -21 -28 -35 -42 -49]
 [-28 -21 -14 -10  -3 -10 -17 -24 -31 -35 -42]
 [-35 -28 -21 -14 -10  -3 -10 -17 -24 -31 -38]
 [-42 -35 -28 -21 -17 -10  -3 -10 -17 -24 -31]
 [-49 -42 -35 -28 -24 -17 -10  -3 -10 -17 -24]
 [-56 -49 -42 -35 -28 -24 -17 -10  -6 -10 -17]
 [-63 -56 -49 -42 -35 -31 -24 -17 -13  -9 -10]]
-10


In [104]:
#SEED AND EXTEND

In [105]:
def reverseComplement(read):
    reversedRead = []
    for i in reversed(range(len(p))):
        if read[i] == 'C':
            reversedRead.append('G')
        elif read[i] == 'G':
            reversedRead.append('C')
        elif read[i] == 'A':
            reversedRead.append('T')
        elif read[i] == 'T':
            reversedRead.append('A')
    return reversedRead

In [121]:
def seedExtend(referenceGenome, readId, isReverseComplement, read, seedLength, margin):
    ''' referenceGenome, readId, read, seedLength, margin, aligner, fmIndex'''
    results = []
    seedPattern = read[0:seedLength]
    seedPositions = findPattern(referenceGenome,seedPattern)
    
    i = 0
    for position in seedPositions:
        # print(position)
        start = position + seedLength
        end = start + len(read) - seedLength + margin # end is not counted in slicing
        if end >= len(referenceGenome):
            continue
            
        alignedRef = referenceGenome[start:end]
        alignedRead = read[seedLength:]
        
        D, alignmentScore = globalAlignment(alignedRef, alignedRead, scoringMatrix)
        alignment, transcript = traceback(alignedRef, alignedRead, D,scoringMatrix)
        
        # print('alignment completed for ' + str(i))
        
        i += 1
        
        results.append((readId, isReverseComplement, position, alignmentScore, transcript))
    
    # print('Completed')
    results.sort(key=lambda x: x[3], reverse=True)
    return results

In [127]:
seedExtend('abaabaabaabaabaabaabaaba$',1,False,'abaabaabaabaabaabb',3,2)

[24, 23, 20, 17, 14, 11, 8, 5, 2, 21, 18, 15, 12, 9, 6, 3, 0, 22, 19, 16, 13, 10, 7, 4, 1]
abbbbbbbbaaaaaaa$aaaaaaaa
{'a': 16, 'b': 8, '$': 1}
{'$': 0, 'a': 1, 'b': 17}
[(0, 'a'), (0, 'b'), (1, 'b'), (2, 'b'), (3, 'b'), (4, 'b'), (5, 'b'), (6, 'b'), (7, 'b'), (1, 'a'), (2, 'a'), (3, 'a'), (4, 'a'), (5, 'a'), (6, 'a'), (7, 'a'), (0, '$'), (8, 'a'), (9, 'a'), (10, 'a'), (11, 'a'), (12, 'a'), (13, 'a'), (14, 'a'), (15, 'a')]
{'$': 0, 'a': 0, 'b': 0}
{'$': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}, 'a': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}, 'b': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}}
a
b
b
b
b
b
b
b
b
a
a
a
a
a
a
a


[(1, False, 3, -14, 'MMMMMMMMMMMMMMDDM'),
 (1, False, 0, -14, 'MMMMMMMMMMMMMMDDM')]

In [128]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Marina\anaconda3\python.exe -m pip install --upgrade pip' command.


In [171]:
from Bio import SeqIO
from Bio.Seq import Seq
import os
import pandas as pd


In [154]:
def parseFile(file_name):
    
    if not os.path.exists(file_name):
        print('File not found')
        
    file_type = file_name[file_name.find('.')+1:]
    
    arr = []
    for seq_record in SeqIO.parse(file_name, file_type):
        #print(seq_record.id)
        #print(repr(seq_record.seq))
        #print(len(seq_record))
        arr.append([seq_record.id,seq_record.seq])
    return arr


In [179]:
def main(match, mismatch, gap, seedLength, margin, fasta, fastq):
    reference = parseFile(fasta)[0][1]
    reads = parseFile(fastq)
    #print(len(reads))
    #print(reference)
    #print(reads)
    
    initFM(reference)

    initScoringMatrix(match, mismatch, gap) 
    
    i = 1

    step = 20
    bestResults = []
    for read in reads:
        if i % step == 0:
            print("Start datetime for read ")
            print(datetime.now())

        
        results = seedExtend(referenceGenome, read[0], False, reverseRead, seedlength, margin)
        if (len(results) > 0):
            bestResults.append(results[0]) 
                    
        
        reverseRead = reverseComplement(read[1])
        
        results = seedExtend(referenceGenome, read[0], True, reverseRead, seedlength, margin)
        if (len(results) > 0):
            bestResults.append(results[0])
            
        
        i += 1
    
    df = pd.DataFrame(bestResults, columns =['read_id', 'is_rev_comp', 'position','alignment_score','transcript'])
    df.to_csv('results/results' + str(match) + str(mismatch) + str(gap) + '.csv')

In [180]:
fasta = 'reference.fasta'
fastq = 'reads.fastq'

seedLength = 10
margin = 2
    
for match in range(1,3):
    for mismatch in range(-3,-1):
         for gap in range(-7, -4):
                main(match, mismatch, gap,seedLength,margin,fasta,fastq)

KeyboardInterrupt: 

In [181]:
fasta = 'reference.fasta'
fastq = 'reads.fastq'
seedLength = 10
margin = 2
main(1, -3, -7,seedLength,margin,fasta,fastq)

MemoryError: 